## Whisper

## Demo - We'll build a simple p2p chat app using the Ethereum Whisper Protocol

![alt text](https://www.easyeth.com/wp-content/uploads/2016/11/Web-3.0-Tech-Stack.jpg "Logo Title Text 1")

## What is Whisper? 

![alt text](https://blog.ethereum.org/wp-content/uploads/2014/08/ethereum-protocols.png "Logo Title Text 1")

- Whisper is a communication protocol for DApps to communicate with each other. (Darkness at a high cost) 
- Off-chain
- Can be considered a Distributed Hash Table or a datagram messaging system

![alt text](https://upload.wikimedia.org/wikipedia/commons/thumb/9/98/DHT_en.svg/500px-DHT_en.svg.png "Logo Title Text 1")

![alt text](https://mainframe.com/static/dark-routing-diagram-1-en.3434e370.png "Logo Title Text 1")

- Every message delivered to every node
- Node sends message to neighboring nodes, but only recipient has the key to decrypt it. All nodes continously forward the message to nearby nodes until the recipient recives it. Then the recipient even forwards it (untraceability) 
- It's an identity-based messaging system, designed for easy and efficient broadcasting, and also for low-level asynchronous communications (publish/subscribe)
- At its most secure mode of operation, Whisper can theoretically deliver 100% darkness. 
- Tradeoff between level of privacy desired and performance
- Included by default in Ethereum’s peer-to-peer (P2P) protocol package (go-ethereum/p2p), which enables users to send messages to others via the same network of machines that comprise Ethereum’s network.
- In order to prevent a DDoS attack, a proof-of-work (PoW) algorithm is used. 
- Messages will be processed (and forwarded further) only if their PoW exceeds a certain threshold, otherwise they will be dropped.

## Use Cases

![alt text](https://cdn-images-1.medium.com/max/1600/1*1r_KRJqVR3O7DGHk1ksXYQ.png "Logo Title Text 1")

- Dapp needs to publish small amounts of non-persistent information (with preset TTL)
- Your Dapps need to communicate with each other prior to actually sending a transaction (i.e settling a deal)
- You have a Dapp that provides non-realtime updates (like the weather)
- Status (messaging app + mobile browser) for connecting to Ethereum raised 44 million in minutes from ICO

## Existing Solutions

![alt text](https://solvethenetwork.com/wp-content/uploads/2016/10/udp.gif "Logo Title Text 1")

- UDP: Similar in API-level, native multicasting. No TTL, security or privacy safeguards.
- 0MQ: A distributed messaging system, no inherent privacy safeguards.
- Bitmessage: (e-mail replacement, only "several thousand/day", larger mails), fixed TTL and no hinting to optimise for throughput
- TeleHash: Secure connection-orientated RTC comms.  Uses DHT to do deterministic routing therefore insecure against simple statistical packet-analysis attacks against a large-scale attacker. Connection oriented, so no TTL and not designed for asynchronous data publication.
- Tox: Higher-level (IM & AV chat) replacement.

## Whisper Features

![alt text](https://image.slidesharecdn.com/soe7omxqegqjgt8ifeda-signature-8b698f217173cb772127f55845adfd529028d4ec3e831e495772f46ba1ada5a8-poli-141130114940-conversion-gate02/95/whisper-22-638.jpg?cb=1417348237 "Logo Title Text 1")

- Whisper API is only exposed to contracts, never to user accounts.
- Low-bandwidth: Only designed for smaller data transfers.
- Unpredictabe latency: Not designed for real-time communication of data.
- Low-bandwidth Not designed for large data transfers.
- Dark (No reliable methods for tracing packets)
- All API for whisper is contained in the web3.shh object. 
- Type it in the interactive console to get a list of all functions and variables provided.

```javascript
var f = web3.shh.filter({topics: ["qwerty"]})
f.get()
web3.shh.getMessages("qwerty")
web3.shh.post({topics: ["qwerty"], payload: "0x847a786376", ttl: "0x1E", workToProve: "0x9" })
```

## Architecture

![alt text](https://davidburela.files.wordpress.com/2016/11/image_thumb3.png?w=1550&h=1168 "Logo Title Text 1")

#### Messages:

- Each message accompanied by a unique process of message sealing, compression and encryption
- Each message has a lifespan (maximum 2 days), a recipient (recognized by a public key) and a group of message topics.
- Whenever you want to send message over the whisper network you need to prove to network you've done some significant work for sealing the message and thus the more work you put in to sealing the message the higher the priority the message will have when propagating it over the network.
- Whisper's Proof of Work consists of a simple SHA3 algorithm in which we try to find the smallest number within a given time frame. 
- Giving the algorithm more time will result in a smaller number which means the message has a higher priority in the network.
- Messages are also sealed with a Time To Live. Peers will automatically flush out messages which have exceeded their time to live (max 2 days) 
- messages may also contain a recipient (public key) and a set of topics. 
- Topics will allow us to specify messages their subject (e.g., "shoes", "laptop", "marketplace", "chat")

Sending a whisper message requires you to:

1. create a new whisper.Message
2. Seal it (optionally encrypt, sign and supply with topics)
3. Send it to your peers

```javascript
topics := TopicsFromString("my", "message")
msg := whisper.NewMessage([]byte("hello world"))  // 1
envelope := msg.Seal(whisper.Opts{                // 2
        From:   myPrivateKey, // Sign it
        Topics: topics,
})
whisper.Send(envelope)                            // 3
```

Watching for specific messages on the whisper network can be done using the Watch method. You have the option to watch for messages from a specific recipient, with specific topics or messages directly directed to you.

```javascript
topics := TopicsFromString("my", "message")
whisper.Watch(Filter{
        Topics: topics
        Fn:     func(msg *Message) {
                fmt.Println(msg)
        },
})
```

#### Sealing (Proof of Work)

![alt text](https://ethereumbuilders.gitbooks.io/guide/content/en/vitalik-diagrams/multikdf.png "Logo Title Text 1")

- Internally, sealing involves hasing contents of the message repeatedly into the smallest possible number via PoW
- The cost of computing PoW can be regarded as the price you pay for allocated resources if you want the network to store your message for a specific time (TTL aka Time to Live)
-  After creating the Envelope, its Nonce should be repeatedly incremented, and then its hash should be calculated. This procedure could be run for a specific predefined time, looking for the lowest hash.
- PoW is defined as average number of iterations, required to find the current BestBit (the number of leading zero bits in the hash), divided by message size and TTL:

```javascript
    PoW = (2^BestBit) / (size * TTL)
```
- we can use PoW as a single aggregated parameter for the message rating.
- What this means to you as a developer is that if you want your messages to have priority over others on the network, you’ll need to spend more computational resources “sealing” them. 
- You can specify this through the `work` parameter when posting a new message. 
- The more work that you perform locally, the faster your message will propagate through the network.

#### Encryption

![alt text](https://cdn-images-1.medium.com/max/640/1*23RpkZuWAeSP7x0YdMtsdQ.png "Logo Title Text 1")

- Every Whisper message must be encrypted either symmetrically or asymmetrically. 
- Messages could be decrypted by anyone who possesses the corresponding key.
- Every node may possess multiple symmetric and asymmetric keys. 
- Upon Envelope receipt, the node should try to decrypt it with each of the keys, depending on Envelope's encryption mode -- symmetric or asymmetric. 
- In case of success, decrypted message is passed to the corresponding Ðapp. In any case, every Envelope should be forwarded to each of the node's peers.
- Asymmetric encryption uses the standard Elliptic Curve Integrated Encryption Scheme with SECP-256k1 public key. 
- Symmetric encryption uses AES GCM algorithm with random 96-bit nonce. 

#### Decryption

- Decryption takes place using a private key, if the message’s envelope was encrypted by no more than one sender.
- In such case, the known full message topic is matched to one of the envelope’s abridged topics; 
- the index is determined and section of the encryption key is decrypted at the beginning of the data segment, so that the final key is retrieved.
